## Part 1: Preprocessing

In [22]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
#from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [23]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [24]:
# Create y_df with the Attrition and Department columns

y_df= attrition_df[['Attrition', 'Department']]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [25]:
# Create a list of at least 10 column names to use as X data
selected_coulums = ['Education', 'Age','DistanceFromHome', 'JobSatisfaction', 'OverTime',
                   'StockOptionLevel','WorkLifeBalance', 'YearsAtCompany',
                   'YearsSinceLastPromotion', 'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[selected_coulums]

# Show the data types for X_df
print(X_df.dtypes)

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [26]:
# Split the data into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df,y_df,test_size=0.2, random_state=42)


In [27]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'] = X_train['OverTime'].map({'Yes':1, 'No':0})
X_test['OverTime'] = X_test['OverTime'].map({'Yes':1, 'No':0})

In [28]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [29]:
from sklearn.preprocessing import  OneHotEncoder
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
dept_encoded = dept_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
train_dept_encoded = dept_encoder.transform(y_train[['Department']])
test_dept_encoded = dept_encoder.transform(y_test[['Department']])


In [30]:
## Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoded = attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])

## Create, Compile, and Train the Model

In [31]:
# Find the number of columns in the X training data
num_columns = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [32]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer

department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)
# Create the output layer

dept_output = layers.Dense(3, activation='softmax', name='dept_output')(department_hidden)


In [33]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrtion_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [34]:
# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'dept_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'dept_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_features (InputLayer  [(None, 10)]                 0         []                            
 )                                                                                                
                                                                                                  
 dense_2 (Dense)             (None, 64)                   704       ['input_features[0][0]']      
                                                                                                  
 dense_3 (Dense)             (None, 32)                   2080      ['dense_2[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   1056      ['dense_3[0][0]']       

In [35]:
model.fit(
    X_train_scaled,
    {'dept_output':train_dept_encoded, 'attrition_output':train_attrition_encoded},
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/100
30/30 [==============================] - 2s 15ms/step - loss: 1.7183 - dept_output_loss: 1.0632 - attrition_output_loss: 0.6551 - dept_output_accuracy: 0.4138 - attrition_output_accuracy: 0.6840 - val_loss: 1.4658 - val_dept_output_loss: 0.8858 - val_attrition_output_loss: 0.5800 - val_dept_output_accuracy: 0.6314 - val_attrition_output_accuracy: 0.7966
Epoch 2/100
30/30 [==============================] - 0s 5ms/step - loss: 1.3180 - dept_output_loss: 0.8151 - attrition_output_loss: 0.5028 - dept_output_accuracy: 0.6553 - attrition_output_accuracy: 0.8404 - val_loss: 1.3298 - val_dept_output_loss: 0.8119 - val_attrition_output_loss: 0.5179 - val_dept_output_accuracy: 0.6314 - val_attrition_output_accuracy: 0.7966
Epoch 3/100
30/30 [==============================] - 0s 5ms/step - loss: 1.2165 - dept_output_loss: 0.7759 - attrition_output_loss: 0.4406 - dept_output_accuracy: 0.6553 - attrition_output_accuracy: 0.8404 - val_loss: 1.2995 - val_dept_output_loss: 0.8055 - val_att

In [36]:
results = model.evaluate(X_test_scaled, {

        'dept_output': test_dept_encoded,
        'attrition_output': test_attrition_encoded,
            },
            verbose = 1)
print (results)

10/10 [==============================] - 0s 3ms/step - loss: 2.7845 - dept_output_loss: 1.7726 - attrition_output_loss: 1.0119 - dept_output_accuracy: 0.5612 - attrition_output_accuracy: 0.7891
[2.7845375537872314, 1.7726376056671143, 1.0119001865386963, 0.5612244606018066, 0.7891156673431396]


In [37]:
# Print the accuracy for both department and attrition
print(f"Department preditions accuracy: {results[3]}")
print(f"Attrition predictions accuracy: {results[4]}")

Department preditions accuracy: 0.5612244606018066
Attrition predictions accuracy: 0.7891156673431396


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.